# Binder equilibration criterion testing
Notebook used to test equilibration on data produced in parallel independent runs using the equilibration condition defined in p.34 of Binder, K. (2010). Monte Carlo Simulation in Statistical Physics An Introduction (5th ed. 2010..). Springer Berlin Heidelberg.

Using $\tau$ to denote the integrated non-linear relaxation time, $t_{M_0}$ to denote the initial time used to calculate observables.

Binder's requires $\tau \ll t_{M_0}$, so a choice of values is considered self consistent when $t_{M0} = 10\tau$ and 
$$\phi^{nl}(t):=\frac{\left< A (t)\right>_T - \left< A (\infty)\right>_T}{\left< A (0)\right>_T - \left< A (\infty)\right>_T}$$
$$\left< A (\infty)\right>_T = \frac{1}{n_{\text{run}}} \sum_{l=1}^{n_{\text{run}}} \frac{1}{M-M_0} \sum_{t=M_0}^{M} A(t,l)$$
($M$ is the total number of MCS)

Last eqn. means that the asymptotic value is calculated by averaging over the entire time series starting from $t_{M_0}$.

#### Key issue:
$m^2(k_{\text{min}})$ relaxes non exponentially to equilibrium!

Therefore $\tau$ accumulates negative values when summing $m^2(k_{\text{min}})$ so that its final value is too small.

In [175]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import glob
import analysis_tools

In [176]:
# parameters:

to_plot = ['Energy']#['Energy','|m|','m2','mk2']
L = '5'
Bex = '0.0'
folderName = 'parallel_test'
mech = ['false']#sys.argv[3]

In [177]:
# creatge table of simulations (just one simulation here)
simulations = analysis_tools.get_simulations(L, folderName, Bex, mech, T=[1.72520])

   L     folderName  Bex   mech         T
0  5  parallel_test  0.0  false  1.725195


In [178]:
# Read data

nonlinear_relax_times = []

# get simulation parameters in namedtuple
sim = list(simulations.itertuples())[0]
path='../analysis/'+sim.folderName+'/table_'+str(sim.L)+'_'+str(sim.L)+'_'+str(sim.Bex)+'_'+str(sim.T)+'_'+str(sim.mech)+'_'+'*'+'.txt'

file_list = glob.glob(path) # list of all files that match the sim parameters

arrays=[]

# iterate over seeds (ind. runs):
for fname in file_list:
    y = analysis_tools.get_table_data_by_fname(fname, print_prog=True)

    arrays.append(y)

all_tables = pd.concat(arrays)

# create new columns w/ abs and squared magnetization (otherwise averages are meaningless)
all_tables['|m|']=all_tables['Magnetization'].abs()
all_tables['m2']=all_tables['Magnetization']**2

# average over independent simulations (all columns)
ind_runs_means = all_tables.groupby(all_tables.index).mean()

# number of runs
n=len(ind_runs_means)

# non-linear relaxation func as defined in (2.57), p.34 of Binder, K. (2010). Monte Carlo Simulation in Statistical Physics An Introduction (5th ed. 2010..). Springer Berlin Heidelberg.
# (mean is taken over the last half of the time series)
nonlinear_relax_func = (ind_runs_means -  ind_runs_means.iloc[int(n/2):].mean())/(ind_runs_means.iloc[0] - ind_runs_means.iloc[int(n/2):].mean())

read ../analysis/parallel_test/table_5_5_0.0_1.7251953058301357_false_4238405562228177337.txt : 65536
read ../analysis/parallel_test/table_5_5_0.0_1.7251953058301357_false_6138320829383856402.txt : 65536
read ../analysis/parallel_test/table_5_5_0.0_1.7251953058301357_false_2975444148659454525.txt : 65536
read ../analysis/parallel_test/table_5_5_0.0_1.7251953058301357_false_8352101331850336645.txt : 65536
read ../analysis/parallel_test/table_5_5_0.0_1.7251953058301357_false_1811066200341819036.txt : 65536
read ../analysis/parallel_test/table_5_5_0.0_1.7251953058301357_false_4026803609728301067.txt : 65536
read ../analysis/parallel_test/table_5_5_0.0_1.7251953058301357_false_2907036230392963128.txt : 65536
read ../analysis/parallel_test/table_5_5_0.0_1.7251953058301357_false_860667096900439879.txt : 65536
read ../analysis/parallel_test/table_5_5_0.0_1.7251953058301357_false_5467741662741665570.txt : 65536
read ../analysis/parallel_test/table_5_5_0.0_1.7251953058301357_false_4651872031572

In [ ]:
# reset nonlinear_relax_func start
nonlinear_relax_func = (ind_runs_means -  ind_runs_means.iloc[int(n/2):].mean())/(ind_runs_means.iloc[0] - ind_runs_means.iloc[int(n/2):].mean())

In [ ]:
# display full dataframe
pd.set_option('display.max_rows', None)

In [179]:
# set observable
observable='Energy'

In [180]:
def find_init_tau(tau_cumsum_to_t_ratio, tol=0.1):
    """find initial tau where the ratio of tau cummulative sum to t (index) is less than tol=0.1 for at least 10 time steps"""
    tau1=tau_cumsum_to_t_ratio.abs().lt(tol).idxmax()
    while not (tau_cumsum_to_t_ratio.abs().loc[tau1:tau1+10] < tol).all():
        tau1=(tau_cumsum_to_t_ratio.abs().loc[tau1+1:] < tol).idxmax()
    return tau1

In [184]:
# find self consistent values for tau and t_{M0}

tau_cumsum_to_t_ratio=(nonlinear_relax_func[observable].cumsum() / nonlinear_relax_func[observable].index)

tau1=find_init_tau(tau_cumsum_to_t_ratio)
nonlinear_relax_func = (ind_runs_means -  ind_runs_means.iloc[tau1:].mean())/(ind_runs_means.iloc[0] - ind_runs_means.iloc[tau1:].mean())
tau_cumsum_to_t_ratio=(nonlinear_relax_func[observable].cumsum() / nonlinear_relax_func[observable].index)
tries=0
while tries < 10 and tau1!=find_init_tau(tau_cumsum_to_t_ratio):
    tries+=1
    tau1=find_init_tau(tau_cumsum_to_t_ratio)
    nonlinear_relax_func = (ind_runs_means -  ind_runs_means.iloc[tau1:].mean())/(ind_runs_means.iloc[0] - ind_runs_means.iloc[tau1:].mean())
    tau_cumsum_to_t_ratio=(nonlinear_relax_func[observable].cumsum() / nonlinear_relax_func[observable].index)
    print(tries)
if tries>=10:
    print('no self consistent value of tau was found!')
else:
    print('After %s tries, found:'%(tries+1))
    print('(tau,t_{M_0}) = ' + str((nonlinear_relax_func[observable].cumsum().loc[tau1],tau1)))

After 1 tries, found:
(tau,t_{M_0}) = (5.798662202521475, 58)


In [ ]:
plt.close('all')

In [ ]:
plt.show()